### Data set: sizes of various flower parts
- given certain parameters, will attempt to predict what flower it is

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

2022-07-21 21:22:47.043032: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-21 21:22:47.043078: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
CSV_COLUMNS = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

train = pd.read_csv(train_path, names=CSV_COLUMNS, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMNS, header=0)

In [4]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [5]:
def input_fn(features, labels, training=True, batch_size=256):
    # convert inputs into dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [6]:
# feature columns
feature_columns = []
for key in train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

In [7]:
print(feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [8]:
# Building the model
# we'll use a classification model, of which there are hundreds of different types
# DNN (deep neural network) classifier

classifier = tf.estimator.DNNClassifier(
    feature_columns = feature_columns,
    # Two hidden layers of 30 and 10 nodes
    hidden_units = [30, 10],
    # 3 different classes to choose from
    n_classes = 3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9r7fbg7f', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


2022-07-21 21:23:02.350556: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-21 21:23:02.350621: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2022-07-21 21:23:02.351071: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# training the model
classifier.train(
    input_fn = lambda: input_fn(train, train_y, training=True),
    steps = 5000
)


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2022-07-21 21:23:06.054042: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-07-21 21:23:06.065303: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT64
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}

	while inferring type of node 'dnn/zero_fraction/cond/output/_18'


INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp9r7fbg7f/model.ckpt.
INFO:tensorflow:/tmp/tmp9r7fbg7f/model.ckpt-0.meta
INFO:tensorflow:100
INFO:tensorflow:/tmp/tmp9r7fbg7f/model.ckpt-0.index
INFO:tensorflow:100
INFO:tensorflow:/tmp/tmp9r7fbg7f/model.ckpt-0.data-00000-of-00001
INFO:tensorflow:100
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 2.3186398, step = 0
INFO:tensorflow:global_step/sec: 182.19
INFO:tensorflow:loss = 1.387519, step = 100 (0.552 sec)
INFO:tensorflow:global_step/sec: 398.937
INFO:tensorflow:loss = 1.1652172, step = 200 (0.248 sec)
INFO:tensorflow:global_step/sec: 434.425
INFO:tensorflow:loss = 1.0287033, step = 300 (0.230 sec)
INFO:tensorflow:global_step/sec: 438.342
INFO:tensorflow:loss = 0.9447596, step = 400 (0.228 sec)
INFO:tensorflow:global_step/sec: 406.224
INFO:tensorflow:loss = 0.90734845, step = 500 (0.247 sec)
INFO:tensorflow

In [10]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test, test_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-07-21T21:23:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9r7fbg7f/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.52650s
INFO:tensorflow:Finished evaluation at 2022-07-21-21:23:19
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.89166665, average_loss = 0.4745093, global_step = 5000, loss = 0.4745093
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmp9r7fbg7f/model.ckpt-5000

Test set accuracy: 0.892



In [14]:
# Actually creating predictions
def input_fn2(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted:")
for feature in features:
    valid = True
    while valid:
        val =  input(feature + ": ")
        if not val.isdigit():
            valid = False
    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn2(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100  * probability)
    )

Please type numeric values as prompted:
SepalLength: 5.1
SepalWidth: 3.3
PetalLength: 1.7
PetalWidth: 0.5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9r7fbg7f/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (77.9%)
